# Using API documentation

With Dartmouth's lookup API, we basically reverse engineered how it works by looking at the request and trying to make sense of it. There is a much better way to learn about the capabilities and requirements of an API: its documentation.

Let's take a look at the documentation of [the weather.gov API](https://www.weather.gov/documentation/services-web-api) for a typical example.


## Pricing

The first thing you want to check is pricing. Many APIs are free, but some come with a price tag. Pricing can be a fixed amount or by usage.

In the case of the weather.gov API, we find the following:

> All of the information presented via the API is intended to be open data, free to use for any purpose. As a public service of the United States Government, we do not charge any fees for the usage of this service, although there are reasonable rate limits in place to prevent abuse and help ensure that everyone has access.

That's great! We do not have to pay to use this service. Because of the rate limit, we should be mindful of how many requests we are making to the API, however.


## Authentication

Some APIs want to know who is making the requests. Any paid API, for example, will have to make sure that the requesting user actually has an active subscription.

The most common way to authenticate yourself is by sending a special header with your requests that includes a special character string called an _API key_ or _token_. This key is assigned to you when you subscribe to the service and is used by the API server to make sure that you are allowed to access the resource you are requesting.

Some more secure APIs may require a more complex authentication process. The documentation will usually tell you what to do to get access.

In the case of the weather.gov API, we do not have to register first. Instead, it uses the `User-Agent` header that we have already seen before to identify the user:

> A User Agent is required to identify your application. This string can be anything, and the more unique to your application the less likely it will be affected by a security event.

So let's set up our user agent:


In [ ]:
user_agent = "(Simon's API demo, simon.stone@dartmouth.edu)"

## The APIs base URL


The next thing you need to know is how to reach the API. Somewhere near the top of the documentation, you should find a base URL.

In case of weather.gov, the API's base URL is: `https://api.weather.gov`


In [ ]:
base_url = "https://api.weather.gov"

## Endpoints

Many APIs offer more than just a single way to interact with the provided data. Every such offering is provided through a separate _endpoint_, which is a path appended to the base URL.

<div class="alert alert-block alert-info">

If you think about it, an API, being an interface, is where _their_ code ends and _your_ code starts. So if you multiple ways to interface with _their_ code, the API has multiple _endpoints_.

</div>

Each endpoint usually has its own section in the documentation. The full explanation of every single endpoint provided by an API is called the _specification_.

For the weather.gov API, we find a separate tab called `"Specification"` that lists all the provided endpoints, the method to interact with them, the required parameters, and the format of the response.

For a very simple example, let's just get a list of all the currently active weather alerts.

From the list, we see that the endpoint `/alerts/active` is the correct one for this request. We could use a lot of different parameters, but none of them are marked as _required_, so we can just go with the defaults instead:


In [ ]:
import requests

endpoint = "/alerts/active"
response = requests.get(base_url + endpoint)

print(response.json())

That is a long list of alerts! We might want to narrow it down to a region of the US.

Studying the list of endpoints, we see two ways to do this:

1. Use the endpoint `/alerts/active` and supply the query parameter `region`
2. use the endpoint `/alerts/active/region/{region}`, where we replace the part `{region}` with the region code

But what are the available regions?


### Navigating endpoints

Finding the right endpoint for your request is the first challenge. Once you have identified the endpoint that can give you the right data, it might require parameters that you have to obtain using other endpoints.

Let's say we were interested in a _weather forecast for Grafton County, NH_.

Scrolling through the specification, we find an endpoint called `/zones/{type}/{zoneId}/forecast`:

> Returns the current zone forecast for a given zone.

But the endpoint requires us to specify `zoneId` and `type`. So how can we find those values for Grafton County?

We could probably look that information up somewhere on in a long list of zones. Usually, however, APIs will over such information through an endpoint, as well!

In this case, we can find the `/zones` endpoint returns a list of all zones with no further information needed:


In [ ]:
endpoint = "/zones"
response = requests.get(base_url + endpoint)

Now that we have this list, we can go through it and look for zones with `Grafton` in their name. Once again, the API documentation can help us navigate the response object to find the right field names:


In [ ]:
for feature in response.json()["features"]:
    if "Grafton" in feature["properties"]["name"]:
        print(feature["properties"])

According to this, there is a zone `"Grafton"` but it does not have any stations associated with it. Instead, the forecast is handled separately for Northern and Southern Grafton. Hanover is in Souther Grafton, so let's go with that:


In [ ]:
type = "public"
zoneId = "NHZ005"
response = requests.get(base_url + f"/zones/{type}/{zoneId}/forecast")
response = response.json()
response["properties"]["periods"]

So there it is: A weather forecast for Southern Grafton. We could now use that API call in some other application to pull always up-to-date information or look up more zones and their forecast by just changing the field names.

Admittedly, this was still quite a bit of work and we had to refer to the API specification quite a bit. Fortunately, many web APIs are also available as Python packages, where all these HTTP requests are wrapped in a more convenient Pythonic interface. In [the next notebook](05-python_wrapped_apis.ipynb), we will look at an example of such a library!


<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
